In [ ]:
import numpy as np
import xgboost as xgb
import pandas as pd
import os
import time
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from xgboost import plot_importance

In [ ]:
#was not able to install essentia on windows but got it running under ubuntu
import essentia.standard as es

In [ ]:
def extract_features_from_file(file):
    features, features_frames = es.MusicExtractor(lowlevelStats=['mean', 'stdev'],
                                                  rhythmStats=['mean', 'stdev'],
                                                  tonalStats=['mean', 'stdev'])(file)
    
    #exclude unnecessary features and features with length vary bpm
    exclude = ["rhythm.beats_position",
               "rhythm.beats_count",
               "metadata.audio_properties.analysis.downmix",
               "metadata.audio_properties.codec",
               "metadata.audio_properties.md5_encoded",
               "metadata.tags.file_name",
               "metadata.version.essentia",
               "metadata.version.essentia_git_sha",
               "etadata.version.extractor",
               "tonal.chords_key",
               "tonal.chords_scale",
               "tonal.key_edma.key",
               "tonal.key_edma.scale",
               "tonal.key_krumhansl.key",
               "tonal.key_krumhansl.scale",
               "onal.key_temperley.key",
               "tonal.key_temperley.scale",
               "metadata.version.extractor",
               "tonal.key_temperley.key"]
    
    features_names = [feature for feature in features.descriptorNames() if feature not in exclude]
    
    #flatten the feature list to get a feature vector
    feature_list = []
    for feature_name in features_names:
        feature = features[feature_name]
        if (type(feature) is np.ndarray):
            feature_list += feature.flatten().tolist()
        else:
            feature_list.append(feature)
    
    return feature_list

In [ ]:
# get the names of all features
def create_feature_namelist(file):
    features, features_frames = es.MusicExtractor(lowlevelStats=['mean', 'stdev'],
                                                  rhythmStats=['mean', 'stdev'],
                                                  tonalStats=['mean', 'stdev'])(file)
    
    exclude = ["rhythm.beats_position",
               "rhythm.beats_count",
               "metadata.audio_properties.analysis.downmix",
               "metadata.audio_properties.codec",
               "metadata.audio_properties.md5_encoded",
               "metadata.tags.file_name",
               "metadata.version.essentia",
               "metadata.version.essentia_git_sha",
               "etadata.version.extractor",
               "tonal.chords_key",
               "tonal.chords_scale",
               "tonal.key_edma.key",
               "tonal.key_edma.scale",
               "tonal.key_krumhansl.key",
               "tonal.key_krumhansl.scale",
               "onal.key_temperley.key",
               "tonal.key_temperley.scale",
               "metadata.version.extractor",
               "tonal.key_temperley.key"]
    
    features_names = [feature for feature in features.descriptorNames() if feature not in exclude]
    
    name_list = []
    i = 0 
    feature_list = []
    for feature_name in features_names:
        feature = features[feature_name]
        if (type(feature) is np.ndarray):
            feature_list += feature.flatten().tolist()
        else:
            feature_list.append(feature)
        while len(name_list) < len(feature_list):
            name_list.append((i, feature))
            i += 1
    
    return name_list

In [ ]:
#test if feature extraction works
audio_file = "data/Dark_Forest_clips/03 - Dohm & Schizoid Bears - Modulation Manipulation- Clip2.wav"

start = time.time()
features = extract_features_from_file(audio_file)
stop = time.time()
print(stop - start)
len(features)
#print(features)

In [ ]:
#get the names of the features
audio_file = "data/Dark_Forest_clips/03 - Dohm & Schizoid Bears - Modulation Manipulation- Clip2.wav"
feature_name_list = create_feature_namelist(audio_file)
feature_name_list

In [ ]:
#get features for all clips
CLIP_FOLDERS = ["data/Dark_Forest_clips/", "data/Full_On_clips/", "data/Goa_clips/", "data/Hi_Tech_clips/"]

feature_data = []
y = []
i = 0

for folder in CLIP_FOLDERS:
    filenames = os.listdir(folder)
#     filenames = filenames[:10]
    for file in filenames:
        print(i)
        print(file)
#         y.append(CLIP_FOLDERS.index(folder))
#         feature = extract_features_from_file(folder + file)
#         feature_data.append(feature)
#         print(len(feature))
        i += 1
#         if (i%10 == 0):
#             print(i)

#create Data Frame to work on
X_to_save = pd.DataFrame(feature_data)
y = pd.DataFrame(y)

#duplicate Data Frame to work on
X_org = X_to_save

In [ ]:
# store to csv
X_to_save.to_csv("essentia_features.csv", index=False)
y.to_csv("genre.csv", index=False)

In [ ]:
#load from csv
X_org = pd.read_csv("essentia_features.csv")
y_org = pd.read_csv("genre.csv")

In [ ]:
#zero mena and unit variance
scaler = StandardScaler()
X = scaler.fit_transform(X_org)
y = np.array(y_org)

### delete all clips of the first two songs of each genre
validation_set_no = np.arange(0, 30) 
validation_set_no = np.concatenate((validation_set_no, np.arange(157,187)))
validation_set_no = np.concatenate((validation_set_no, np.arange(320,355)))
validation_set_no = np.concatenate((validation_set_no, np.arange(495,521)))

X_final_validation = np.take(X, validation_set_no, 0)
X = np.delete(X, validation_set_no, 0)

y_final_validation = np.take(y, validation_set_no, 0)
y = np.delete(y, validation_set_no, 0)

### reduce feature vector
#X = X[:,:200]

### delete the best ten features
# X = np.delete(X, (867, 96, 875, 119, 347, 871, 475, 173, 613, 75), 1)
# X_final_validation = np.delete(X_final_validation, (867, 96, 875, 119, 347, 871, 475, 173, 613, 75), 1)

### only take the best ten features
#X = np.take(X, (867, 96, 875, 119, 347, 871, 475, 173, 613, 75), 1)
# X_final_validation = np.take(X_final_validation, (867, 96, 875, 119, 347, 871, 475, 173, 613, 75), 1)

### plot shape of X
print(X.shape)

### split Output and Input data into train and test data (data also gets shuffeld)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [ ]:
#tran XGBoost
D_train = xgb.DMatrix(X_train, label=y_train)
D_test = xgb.DMatrix(X_test, label=y_test)

param = {
    'eta': 0.3, 
    'max_depth': 2,
    'objective': 'multi:softprob',  
    'num_class': 4} 

steps = 20  # The number of training iterations

model = xgb.train(param, D_train, steps)

In [ ]:
#plot accuarcy
from sklearn.metrics import precision_score, recall_score, accuracy_score

preds = model.predict(D_test)
best_preds = np.asarray([np.argmax(line) for line in preds])

print("Precision = {}".format(precision_score(y_test, best_preds, average='macro')))
print("Recall = {}".format(recall_score(y_test, best_preds, average='macro')))
print("Accuracy = {}".format(accuracy_score(y_test, best_preds)))

In [ ]:
#display confusion matrix
labels = ["Dark Forest", "Full On", "Goa", "Hi Tech"]
fig = plt.figure(figsize=(8, 6));
ax = fig.add_subplot(111);
cax = ax.matshow(cm);

for (i, j), z in np.ndenumerate(cm):
    ax.text(j, i, '{:0.2f}'.format(z), ha='center', va='center',
            bbox=dict(boxstyle='round', facecolor='white', edgecolor='0.3'))

plt.title('Confusion matrix of the classifier');
ax.set_xticklabels([" "] + labels);
ax.set_yticklabels([" "] + labels);
plt.xlabel('Predicted');
plt.ylabel('True');
plt.show();

### Final validation use with caution

In [ ]:
from sklearn.metrics import precision_score, recall_score, accuracy_score

D_validation = xgb.DMatrix(X_final_validation)

preds = model.predict(D_validation)
best_preds = np.asarray([np.argmax(line) for line in preds])

print("Precision = {}".format(precision_score(y_final_validation, best_preds, average='macro')))
print("Recall = {}".format(recall_score(y_final_validation, best_preds, average='macro')))
print("Accuracy = {}".format(accuracy_score(y_final_validation, best_preds)))

In [ ]:
#display confusion matrix
labels = ["Dark Forest", "Full On", "Goa", "Hi Tech"]
fig = plt.figure(figsize=(8, 6));
ax = fig.add_subplot(111);
cax = ax.matshow(cm);

for (i, j), z in np.ndenumerate(cm):
    ax.text(j, i, '{:0.2f}'.format(z), ha='center', va='center',
            bbox=dict(boxstyle='round', facecolor='white', edgecolor='0.3'))

plt.title('Confusion matrix of the classifier');
ax.set_xticklabels([" "] + labels);
ax.set_yticklabels([" "] + labels);
plt.xlabel('Predicted');
plt.ylabel('True');
plt.show();

In [ ]:
#plot the most important features
plot_importance(model, max_num_features=10);

In [ ]:
feature_name_list = pd.read_csv("feature_names.csv")
feature_name_list.loc[867]